In [9]:
# Import required libraries

import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import re
# from collections import defaultdict
# import geopandas
# import seaborn as sns

# sns.set()

In [11]:
# Load data from Excel file
df = r'C:\Users\Mani_Kandan_Raja\Desktop\Predictive Modeling for Revenue Growth\online+retail+ii\online_retail_II.xlsx'

# Load each sheet into separate dataframes
df_0910 = pd.read_excel(df, sheet_name='Year 2009-2010')
df_1011 = pd.read_excel(df, sheet_name='Year 2010-2011')

# Concatenate the two dataframes
data = pd.concat([df_0910, df_1011], ignore_index=True)

# Display the first few rows of the concatenated dataframe
print(data.head())


  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


In [12]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB


### Observations

'Description' and 'CustomerID' columns have null values.

Lets check if we can fill CustomerID from InvoiceNo.

In [14]:
# Check one-to-many mapping of some columns to check for data correctness

def isOneToMany(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    print(first == 1)

In [16]:
# Check if each InvoiceNo is unique to a CustomerID
isOneToMany(data, 'Invoice', 'Customer ID')

True


Considering we are not able to fill up CustomerID, and we have enough data, we will drop observations with NaN CustomerID

In [21]:
# Delete rows with missing CustomerID
data = data.drop(data[data['Customer ID'].isnull()].index).reset_index(drop=True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824364 entries, 0 to 824363
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      824364 non-null  object        
 1   StockCode    824364 non-null  object        
 2   Description  824364 non-null  object        
 3   Quantity     824364 non-null  int64         
 4   InvoiceDate  824364 non-null  datetime64[ns]
 5   Price        824364 non-null  float64       
 6   Customer ID  824364 non-null  float64       
 7   Country      824364 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 50.3+ MB


Delete missing CustomerId has removed all missing Description rows too.

In [26]:
# Convert "Invoice" to a string type series
data['Invoice'] = data['Invoice'].astype('str')

In [27]:
# Convert "Description" to a string type series and remove extra whitespaces
data['Description'] = data.Description.astype('str')
data['Description'] = data.Description.str.strip()

In [28]:
# Check for data correctness based on one to many relationship between some columns

isOneToMany(data,'Invoice','Country')
isOneToMany(data,'StockCode','Description')

True
False


StockCode and Description should have one-to-many relationship. Which means each StockCode needs to uniquely represent an item Description, but each Description may not have just one StockCode. But our dataset has multiple Description for same StockCode.

Let us analyze this:

In [31]:
data

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
824359,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
824360,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
824361,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
824362,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [34]:
# Check duplicacy of description in StockCode

products = data.groupby(['StockCode','Description'],as_index=False).count()
products = products.loc[:,['StockCode','Description']]
stockCount = products['StockCode'].value_counts().to_frame()
stockList = list(stockCount[stockCount['count']>1].index) #List of stockCode with more than 1 Description
products[products['StockCode'].isin(stockList)].head(10)

,StockCode,Description
17,16012,FOOD/DRINK SPONGE STICKERS
18,16012,FOOD/DRINK SPUNGE STICKERS
62,20615,BLUE POLKADOT PASSPORT COVER
63,20615,BLUE SPOTTY PASSPORT COVER
70,20622,VIP PASSPORT COVER
71,20622,VIPPASSPORT COVER
72,20652,BLUE POLKADOT LUGGAGE TAG
73,20652,BLUE SPOTTY LUGGAGE TAG
78,20658,RED RETROSPOT LUGGAGE TAG
79,20658,RED SPOTTY LUGGAGE TAG


Each StockCode does identify unique item, but there are some data entry errors in the description.

In [43]:
# Remove duplicate entries
products.drop_duplicates('StockCode', keep='first', inplace=True)

In [44]:
# Correct the mapping of StockCode and Description

df = pd.merge(data,products,on='StockCode', how='left',validate='many_to_one')
df.drop('Description_x',axis=1,inplace=True)
df.rename(columns={'Description_y':'Description'},inplace=True)
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
0,489434,85048,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,PINK CHERRY LIGHTS
2,489434,79323W,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,WHITE CHERRY LIGHTS
3,489434,22041,48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX


In [45]:
# Check for StockCode and Description mapping
isOneToMany(df,'StockCode','Description')

True


The data description in the data source mentions that the StockCode is a 5 digit integer. But we observed StockCodes with length different from 5 digits. We will analyse them to check for any discrepancies.

In [46]:
df[(df.StockCode.str.len())==6].sort_values(by='StockCode').head(10)

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
8679,490323,10123C,1,2009-12-04 15:27:00,0.65,14087.0,United Kingdom,HEARTS WRAPPING TAPE
235717,518505,10123C,144,2010-08-09 13:10:00,0.06,14277.0,France,HEARTS WRAPPING TAPE
43848,495053,10123C,12,2010-01-20 14:36:00,0.65,17351.0,United Kingdom,HEARTS WRAPPING TAPE
97148,501806,10123C,10,2010-03-19 14:02:00,0.65,17673.0,United Kingdom,HEARTS WRAPPING TAPE
406896,536863,10123C,1,2010-12-03 11:19:00,0.65,17967.0,United Kingdom,HEARTS WRAPPING TAPE
69087,498364,10123C,12,2010-02-18 13:50:00,0.65,16170.0,United Kingdom,HEARTS WRAPPING TAPE
252736,520584,10123C,12,2010-08-27 10:40:00,0.65,12839.0,United Kingdom,HEARTS WRAPPING TAPE
270374,522773,10123C,12,2010-09-16 14:34:00,0.65,12369.0,Austria,HEARTS WRAPPING TAPE
252495,520564,10123C,5,2010-08-26 17:11:00,0.65,17402.0,United Kingdom,HEARTS WRAPPING TAPE
271802,522917,10123C,12,2010-09-17 14:10:00,0.65,12425.0,Portugal,HEARTS WRAPPING TAPE


In [47]:
df[(df.StockCode.str.len())<5].sort_values(by='StockCode').head(10)

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
442269,539473,C2,1,2010-12-19 14:24:00,50.0,14911.0,EIRE,CARRIAGE
274427,523082,C2,1,2010-09-20 12:15:00,50.0,14911.0,EIRE,CARRIAGE
698242,569739,C2,1,2011-10-06 10:48:00,50.0,14911.0,EIRE,CARRIAGE
269456,522642,C2,1,2010-09-16 10:29:00,50.0,13374.0,United Kingdom,CARRIAGE
700909,570008,C2,1,2011-10-07 09:30:00,50.0,13359.0,United Kingdom,CARRIAGE
265546,522121,C2,1,2010-09-13 09:27:00,50.0,14911.0,EIRE,CARRIAGE
705478,570426,C2,1,2011-10-10 13:55:00,50.0,13352.0,United Kingdom,CARRIAGE
259983,521549,C2,1,2010-09-06 15:55:00,50.0,14156.0,EIRE,CARRIAGE
707816,570651,C2,1,2011-10-11 13:34:00,50.0,14911.0,EIRE,CARRIAGE
708693,570694,C2,1,2011-10-12 08:10:00,50.0,14911.0,EIRE,CARRIAGE


There are some 6-digit codes in StockCodes which consist of 5 digits and last one an alphabet. The last albhabet seems to distinguish the variants of the item specified by 5-digit code. So they are valid.

The StockCodes with lesser than 5 digits seem to be shipping charges/discounts/other incidental charges related to an order. As these are not related to companies sales, we will delete them.

In [48]:
# Drop observations with incidental charges

df = df.drop(df[(df.StockCode.str.len())<5].index).reset_index(drop=True)

In [51]:
df.describe(include='all')

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
count,820746,820746,820746.000000,820746,820746.000000,820746.000000,820746,820746
unique,43995,4639,NaN,NaN,NaN,NaN,41,4602
top,576339,85123A,NaN,NaN,NaN,NaN,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
freq,541,5322,NaN,NaN,NaN,NaN,739943,5322
mean,NaN,NaN,12.460804,2011-01-01 21:18:13.274264832,2.974356,15331.087678,NaN,NaN
min,NaN,NaN,-80995.000000,2009-12-01 07:45:00,0.000000,12346.000000,NaN,NaN
25%,NaN,NaN,2.000000,2010-07-06 11:21:00,1.250000,13982.000000,NaN,NaN
50%,NaN,NaN,5.000000,2010-12-03 14:04:00,1.950000,15270.000000,NaN,NaN
75%,NaN,NaN,12.000000,2011-07-27 15:14:00,3.750000,16799.000000,NaN,NaN
max,NaN,NaN,80995.000000,2011-12-09 12:50:00,649.500000,18287.000000,NaN,NaN


#### Observations

1. Quantity values look incorrect
2. In some cases, the UnitPrice is 0. Some more investigation is needed to check data correctness
3. 89% of the sales is coming from UK
4. Number of unique items for StockCode and Description is different indicating one-to-many relationship.


The CustomerID appears as a float in the dataset. As it is an identifier, we will append the letter C to it to make sure it is not treated like a float/int.

In [54]:
df['Customer ID'] = 'C' + df['Customer ID'].astype('int').astype('str')
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
0,489434,85048,12,2009-12-01 07:45:00,6.95,C13085,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,12,2009-12-01 07:45:00,6.75,C13085,United Kingdom,PINK CHERRY LIGHTS
2,489434,79323W,12,2009-12-01 07:45:00,6.75,C13085,United Kingdom,WHITE CHERRY LIGHTS
3,489434,22041,48,2009-12-01 07:45:00,2.10,C13085,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,24,2009-12-01 07:45:00,1.25,C13085,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX


#### Check large Quantity values for data correctness

In [60]:
# Check negative quanitities

df[(df.Quantity<1) & (~df['Invoice'].str.startswith('C'))]

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description


All the negative quantities are cancelled orders.

In [65]:
# Check for unit quantity of more than 4000
df[(abs(df.Quantity)>12000)]

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
65205,497946,37410,19152,2010-02-15 11:57:00,0.10,C13902,Denmark,BLACK AND WHITE PAISLEY FLOWER MUG
94049,501534,21099,12960,2010-03-17 13:09:00,0.10,C13902,Denmark,SET/6 STRAWBERRY PAPER CUPS
94050,501534,21092,12480,2010-03-17 13:09:00,0.10,C13902,Denmark,SET/6 STRAWBERRY PAPER PLATES
94051,501534,21091,12960,2010-03-17 13:09:00,0.10,C13902,Denmark,SET/6 WOODLAND PAPER PLATES
94052,501534,21085,12744,2010-03-17 13:09:00,0.10,C13902,Denmark,SET/6 WOODLAND PAPER CUPS
453775,541431,23166,74215,2011-01-18 10:01:00,1.04,C12346,United Kingdom,MEDIUM CERAMIC TOP STORAGE JAR
453780,C541433,23166,-74215,2011-01-18 10:17:00,1.04,C12346,United Kingdom,MEDIUM CERAMIC TOP STORAGE JAR
820271,581483,23843,80995,2011-12-09 09:15:00,2.08,C16446,United Kingdom,"PAPER CRAFT , LITTLE BIRDIE"
820272,C581484,23843,-80995,2011-12-09 09:27:00,2.08,C16446,United Kingdom,"PAPER CRAFT , LITTLE BIRDIE"


The two largest quantities of 80995 and 74215 units are part of cancelled orders. The quantity of 12540 has 0 UnitPrice and is the only item in the order. So we will delete it.

In [66]:
df = df.drop(df[df.Quantity==12540].index).reset_index(drop=True)
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description
0,489434,85048,12,2009-12-01 07:45:00,6.95,C13085,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,489434,79323P,12,2009-12-01 07:45:00,6.75,C13085,United Kingdom,PINK CHERRY LIGHTS
2,489434,79323W,12,2009-12-01 07:45:00,6.75,C13085,United Kingdom,WHITE CHERRY LIGHTS
3,489434,22041,48,2009-12-01 07:45:00,2.10,C13085,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE"
4,489434,21232,24,2009-12-01 07:45:00,1.25,C13085,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX


In [63]:
# Check one-to-many mapping between CustomerID and Country

isOneToMany(df, 'Customer ID', 'Country')

False


In the given dataset, some of the CustomerID are linked with 2 countries. As per the data attribute description: 'Country' column is the name of the country where each customer resides. But we don't have any information on how is this data being captured. Is it through IP address of the country while creating account, or may be based on the shipping address, or may be something else.

Logically, each CustomerID should be linked to one country only. The reason for having more than one country could be:
1. Data entry error
2. Customer has moved to another country, and has got the address changed in his account
3. In case this attribute reflects the shipping address, the customer has shipped the order to an address different from his own. 
4. In case this attribute is captured through the IP address while ordering, the customer might be ordering while travelling to another country.

Further analysis of data does not make it clear what is the reason behind 2 countries for a CustomerID, so for now, we are not making any changes in the CustomerID and country linkage.

## Feature engineering

At this stage, we can derive following features from the already existing ones to aid in our analysis.
1. CancelledOrder containing boolean values, 1 if order was cancelled, 0 otherwise
2. InternationalOrders containing boolean values, 0 if order came from UK, 1 if the order came from outside UK
3. TotalPrice containing float values = unitPrice * Quantity

??Avg per unit price?/

grouping of item description



In [68]:
# Add new column with bool value: 0 for not cancelled order, 1 for cancelled order
df['CancelledOrder'] = df.Invoice.str.startswith('C').astype('int')

In [72]:
df['CancelledOrder'].value_counts()

CancelledOrder
0    802773
1     17972
Name: count, dtype: int64

In [73]:
df.Country.value_counts()

Country
United Kingdom          739942
Germany                  16937
EIRE                     15958
France                   13690
Netherlands               5024
Spain                     3703
Switzerland               3005
Belgium                   2973
Portugal                  2433
Australia                 1904
Channel Islands           1652
Italy                     1497
Norway                    1422
Sweden                    1286
Cyprus                    1171
Finland                    999
Austria                    901
Denmark                    792
Greece                     658
Japan                      578
USA                        530
Poland                     529
Unspecified                523
United Arab Emirates       383
Singapore                  332
Israel                     324
Malta                      291
Iceland                    253
Canada                     227
Lithuania                  189
RSA                        122
Brazil                      94


In [74]:
# Delete the countries 'Unspecified' and 'European Community'
df = df.drop(df[df.Country.isin(['Unspecified','European Community'])].index).reset_index(drop=True)

In [75]:
# Add new column 'InternationalOrders' with bool values, 0 for United Kingdom, and 1 for other countries
df['InternationalOrders'] = (df.Country!='United Kingdom').astype('int')

In [77]:
# Add a column for total value for each product in each order
df['TotalPrice'] = df.Quantity*df.Price

In [78]:
df.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,Customer ID,Country,Description,CancelledOrder,InternationalOrders,TotalPrice
0,489434,85048,12,2009-12-01 07:45:00,6.95,C13085,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS,0,0,83.4
1,489434,79323P,12,2009-12-01 07:45:00,6.75,C13085,United Kingdom,PINK CHERRY LIGHTS,0,0,81.0
2,489434,79323W,12,2009-12-01 07:45:00,6.75,C13085,United Kingdom,WHITE CHERRY LIGHTS,0,0,81.0
3,489434,22041,48,2009-12-01 07:45:00,2.10,C13085,United Kingdom,"RECORD FRAME 7"" SINGLE SIZE",0,0,100.8
4,489434,21232,24,2009-12-01 07:45:00,1.25,C13085,United Kingdom,STRAWBERRY CERAMIC TRINKET BOX,0,0,30.0


In [79]:
df.to_csv('Data_Acq.csv')